## Matchups Analysis - Part 1
This notebook is geared at quality assessments of the matchups runs.

In [ ]:
import numpy as np


def value_locate(refx, x):
    """
    VALUE_LOCATE locates the positions of given values within a
    reference array.  The reference array need not be regularly
    spaced.  This is useful for various searching, sorting and
    interpolation algorithms.
    The reference array should be a monotonically increasing or
    decreasing list of values which partition the real numbers.  A
    reference array of NBINS numbers partitions the real number line
    into NBINS+1 regions, like so:
    REF:           X[0]         X[1]   X[2] X[3]     X[NBINS-1]
        <----------|-------------|------|---|----...---|--------------->
    INDICES:  -1           0          1    2       3        NBINS-1
    VALUE_LOCATE returns which partition each of the VALUES falls
    into, according to the figure above.  For example, a value between
    X[1] and X[2] would return a value of 1.  Values below X[0] return
    -1, and above X[NBINS-1] return NBINS-1.  Thus, besides the value
    of -1, the returned INDICES refer to the nearest reference value
    to the left of the requested value.
    
    Example:
    >>> refx = [2, 4, 6, 8, 10]
    >>> x = [-1, 1, 2, 3, 5, 5, 5, 8, 12, 30]
    >>> print value_locate(refx, x)
    array([-1, -1,  0,  0,  1,  1,  1,  3,  4,  4])
    
    
    This implementation is likely no the most efficient one, as there is
    a loop over all x, which will in practice be long. As long as x is
    shorter than 1e6 or so elements, it should still be fast (~sec).
    
    
    """
    print ("TODO: check if refx is monotonically increasing.")
    
    refx = np.array(refx)
    x = np.array(x)
    loc = np.zeros(len(x), dtype='int')
    
    for i in range(len(x)):
        ix = x[i]
        ind = ((refx - ix) <= 0).nonzero()[0]
        if len(ind) == 0:
            loc[i] = -1
        else: loc[i] = ind[-1]
    
    return loc
    
if __name__ =="__main__":
    # test case
    refx = [2, 4, 6, 8, 10]
    x = [-1, 1, 2, 3, 5, 5, 5, 8, 12, 30]
    
    res = value_locate(refx, x)
    assert list(res) == [-1, -1,  0,  0,  1,  1,  1,  3,  4,  4]
    print ("Test(s) passed!")
    
    x= np.random.random(1e6)*20
    res = value_locate(refx, x)
    print ("Done!")

In [356]:
def popcount(a):
    """
    Computes the popcount (Hamming Weight) of a numpy array of 32-bit integers. The popcount of an integer is the 
    number of set bits in an integer. http://en.wikipedia.org/wiki/Hamming_weight
    
    Parameters:
    -----------
    a: array_like, dtype=uint32, shape=(n,m)
      An array of 32-bit integers. Note that the type is not important as long as the values are smaller than 2**32-1
    
    Returns:
    --------
    popcounts: array_like, dtype=uint32, shape=(n,m)
      An array containing the popcount for each element of a
    """
    a -= np.bitwise_and(np.right_shift(a, 1), 0x55555555)
    a = np.bitwise_and(a, 0x33333333) + np.bitwise_and(np.right_shift(a, 2), 0x33333333)
    return np.right_shift(np.bitwise_and((np.bitwise_and(a + np.right_shift(a, 4), 0xF0F0F0F) * 0x1010101), 0xffffffff), 24)

In [1]:
import pandas as pd
import pickle
import glob
import os
import netCDF4 as nc
import numpy as np
import re

In [2]:
def ReturnL2BaseName(splitChar='_',ext='L2'):
    '''Generator. Yields basename to search the matchups table'''
# Building the list of files processed
    l2fiDir = '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/Matchups/L2/AllFiles/'
    l2Ext = '*.%s' % ext
    searchPat = os.path.join(l2fiDir,l2Ext)
    l2FiGenList = glob.iglob(searchPat)
    for l2file in l2FiGenList:
        basename = l2file.split('_')[0].split('/')[-1]
        yield basename, l2file

In [3]:
def GetData(fn,bands=['412','443','490','510','555','670']):
    rrsDict = dict.fromkeys(bands)
    rrsUncDict = dict.fromkeys(bands)
    with nc.Dataset(fn) as ds:
        gpV = ds.groups['geophysical_data'].variables
        for band in bands:
            try:
                rrsDict[band] = gpV['Rrs_%s' % band ][:]
                rrsUncDict[band] = gpV['Rrs_unc_%s' % band][:]
            except KeyError:
                rrsDict, rrsUncDict = None,None
                break
    return rrsDict,rrsUncDict

In [4]:
def GetTarget(df,baseName):
    '''Generator. Yields dataframe indices relevant to a given L2 file.'''
    dfSub = df[df['file_name'].str.contains(baseName)]
    for idx in dfSub.index.values:
        yield idx
        
def GetUncLatLonRng(fn):
    '''Return lat/lon rng, from each file used in unc sim, as lat & lon tuples.'''
    with nc.Dataset(fn) as ds:
        naV = ds.groups['navigation_data'].variables
        latitude = naV['latitude'][:]
        longitude = naV['longitude'][:]
    latRng = (latitude.min(),latitude.max())
    lonRng = (longitude.min(),longitude.max())
    return latRng,lonRng,latitude,longitude

def CheckLocation(mUpLat,mUpLon,uncLatRng,uncLonRng):
    mLat_is_in_rng = False
    mLon_is_in_rng = False
    if mUpLat >= uncLatRng[0] and mUpLat <= uncLatRng[1]:
        mLat_is_in_rng = True
    if mUpLon >= uncLonRng[0] and mUpLon <= uncLonRng[1]:
        mLon_is_in_rng = True
    return mLat_is_in_rng, mLon_is_in_rng    

In [5]:
def GetCntrPxlRowCol(latArray,mUpLat,lonArray,mUpLon):
    latLonDist = abs(latArray - mUpLat) + abs(lonArray - mUpLon)
    i,j = np.unravel_index(latLonDist.argmin(),latLonDist.shape)
    boxValid = (np.array((i,j)) >1).all() and i < (uncLat.shape[0] - 2)  \
                and j < (uncLat.shape[1] - 2)
    return (i,j),boxValid

In [ ]:
def ValidateData(data,boxCntrRowCol,boxSide=5):
    '''
    This assumes SeaWiFS
    data is a dictionary containing Rrs_412...555, and aot_865
    
    Given a box center
    
    '''

In [6]:
def GetStats(rrsDict,rrsUncDict,mUpRrsDict,boxCntrRowCol,filteredMean=True):
    '''
    If filteredMean is True:
    the mean is calculated for points within +/-1.5sigma of the mean of the 
    box where sigma is the stdev of the unfiltered (5x5) box
    '''
    bands = rrsDict.keys()
    subRrsDict = dict.fromkeys(bands)
    subRrsUncDict = dict.fromkeys(bands)
    subMupRrsUncDict = dict.fromkeys(bands)
    i,j = boxCntrRowCol
    boxXRng = [i - 2,i + 3]
    boxYRng = [j - 2,j + 3]
    for band in bands:
        #print(band)
        subRrs = rrsDict[band][boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]]
        subRrsUnc = rrsUncDict[band][boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]]
        try: # QA on Rrs -- TODO: expand QA to mach B&W2006
            if subRrs.compressed().size >= (subRrs.size / 2):
                subRrsDict[band] = subRrs.compressed().mean()
                mUpSatDiff = mUpRrsDict[band] - subRrs.compressed()
                sqrdDiff = np.power(mUpSatDiff,2)
                mUpRrsUnc = np.sqrt(sqrdDiff.sum() / subRrs.compressed().size)
                subMupRrsUncDict[band] = mUpRrsUnc
            else:
                subRrsDict[band] = np.nan
                subMupRrsUncDict[band] = np.nan
        except AttributeError:
            subRrsDict[band] = subRrs.mean()
        try: # QA on Rrs_unc
            if subRrsUnc.compressed().size >= (subRrsUnc.size / 2):
                subRrsUncDict[band] = np.sqrt(np.sum(np.power(subRrsUnc.compressed(),2))) / \
                                                subRrsUnc.compressed().size
            else:
                subRrsUncDict[band] = np.nan
        except AttributeError:
            subRrsUncDict[band]
    return subRrsDict, subRrsUncDict, subMupRrsUncDict

In [7]:
# Load pandas Matchups table
dfName = 'dfMatchUpSWF.pkl'
if os.path.exists(dfName):
    dfMatchups = pd.read_pickle(dfName)
    dfMatchups[dfMatchups==-999] = np.nan
else:
    print('File not found!!')

In [54]:
# get whether a particular index' lat AND lon is within interval: 
    # record idx, basename, uncLatRng, mUpInLatRng, uncLonRng, mUpInLonRng: 
    # => dict #1
# for indices with lat AND lon in range: 
    # record idx, pixel coords in corresponding file,  
    # whether box of 5x5 around pixel coord is possible,
    # and number of valid cells in the 5x5 box.
    # => dict #2
# for validated indices:
    # record idx, basename, unc_Rrs_vvv,  unc_Rrs_unc_vvv
    # note that unc_Rrs_unc_vvv (like unc_Rrs_vvv) is calculated from the 5x5 box 
    # surrounding the pixel coord in dict #2. As such unc_Rrs_unc_vvv is calculated by 
    # propagating the uncertainties of the cells in the 5x5 box.
    # => dict #3


In [8]:
dfMatchups.columns

Index(['cruise_name', 'file_name', 'date_time', 'latitude', 'longitude',
       'seawifs_npixel', 'seawifs_tdiff', 'seawifs_solz', 'seawifs_senz',
       'seawifs_cv', 'seawifs_es_thresh', 'seawifs_windspeed', 'insitu_Rrs412',
       'insitu_Rrs443', 'insitu_Rrs490', 'insitu_Rrs510', 'insitu_Rrs555',
       'insitu_Rrs670', 'seawifs_Rrs412', 'seawifs_Rrs443', 'seawifs_Rrs490',
       'seawifs_Rrs510', 'seawifs_Rrs555', 'seawifs_Rrs670', 'L1_name'],
      dtype='object')

In [12]:
def GetMupRrs(df,idx):
    ser = df.loc[idx,:].filter(regex='insitu')
    lst = ser.index.tolist()
    bandFinder = re.compile('insitu_Rrs([0-9]+)')
    bands = [bandFinder.findall(lbl)[0] for lbl in lst]
    mUpDict = dict.fromkeys(bands)
    for band in bands:
        mUpDict[band] = ser.filter(regex=band).values[0]
    return mUpDict

In [20]:
'''
Run this to construct several dictionaries 
Dict 1 contains summary info about all matchup entries and whether the row is valid, 
i.e. whether both in situ lat/lon are contained in the corresponding UNC. scene.
Dict 2 contains info on the matchup box and whether. 
Dict 3 contains data and stats from matchups & corresponding MC uncertainty runs
Dict 4
'''
dict1Labels = ['ID','uncLatRng','mUpInLatRng','uncLonRng','mUpInLonRng','rowValid']
dict2Labels = ['ID','boxCntrRowCol','boxValid','numValCells']
dict3Labels = ['ID']
bands = ['412','443','490','510','555','670']
for band in bands:
    dict3Labels.append('insitu_Rrs%s' % band)
    dict3Labels.append('seawifs_Rrs%s' % band)
    dict3Labels.append('Rrs_%s' % band)
    dict3Labels.append('Matchup_Rrs_unc_%s' % band)
    dict3Labels.append('Rrs_unc_%s' % band)
summaryLatIn, summaryLonIn, totalEntries = 0,0,0
dict1 = {dcl: [] for dcl in dict1Labels}
dict2 = {dcl: [] for dcl in dict2Labels}
dict3 = {dcl: [] for dcl in dict3Labels}
dict4 = {}
baseNameGen = ReturnL2BaseName()
# go through files
totalFiles = 0
totalRows = 0
for baseName,l2FileName in baseNameGen:
    totalFiles += 1
    dfIdcs = GetTarget(dfMatchups,baseName)
    dict4[baseName] = []
    # go through all rows corresponding to a given file 
    firstPass = True
    for dfIdx in dfIdcs:
        totalRows += 1
        # first task is to figure out whether a valid correspondence exists between
        # matchup data and the relevant l2 file...
        dict1['ID'].append(dfIdx)
        mUpLat = dfMatchups.latitude[dfIdx]
        mUpLon = dfMatchups.longitude[dfIdx]
        uncLatRng, uncLonRng, uncLat, uncLon = GetUncLatLonRng(l2FileName)
        dict1['uncLatRng'].append(uncLatRng)
        dict1['uncLonRng'].append(uncLonRng)
        mUpInLatRng, mUpInLonRng = CheckLocation(mUpLat, mUpLon, uncLatRng, uncLonRng)
        dict1['mUpInLatRng'].append(mUpInLatRng)
        dict1['mUpInLonRng'].append(mUpInLonRng)
        rowValid = mUpInLatRng and mUpInLonRng
        dict1['rowValid'].append(rowValid)
        if rowValid:
            dict2['ID'].append(dfIdx)
            # Get cell closest in lat/lon to corres. mUp lat/lon
            roColTpl,boxVal = GetCntrPxlRowCol(uncLat, mUpLat, uncLon, mUpLon)
            dict2['boxCntrRowCol'].append(roColTpl)
            dict2['boxValid'].append(boxVal)
            if boxVal:
                if firstPass:
                    # Load file data (once even if multiple matchups in the same scene)
                    firstPass = False
                    rrsDict,rrsUncDict = GetData(l2FileName)
                    mUpRrsDict = GetMupRrs(dfMatchups,dfIdx)
                if not rrsDict or not rrsUncDict:
                    continue
                subRrsDict,subRrsUncDict,subMupRrsUncDict = GetStats(rrsDict,
                                                                     rrsUncDict,
                                                                     mUpRrsDict,
                                                                     boxCntrRowCol=roColTpl)
                for band in bands:
                    dict3['insitu_Rrs%s' % band].append(dfMatchups.loc[dfIdx,'insitu_Rrs%s' % band])
                    dict3['seawifs_Rrs%s' % band].append(dfMatchups.loc[dfIdx,'seawifs_Rrs%s' % band])
                    dict3['Rrs_%s' % band].append(subRrsDict[band])
                    dict3['Rrs_unc_%s' % band].append(subRrsUncDict[band])
                    dict3['Matchup_Rrs_unc_%s' % band].append(subMupRrsUncDict[band])
                dict3['ID'].append(dfIdx)
                dict4[baseName].append(dfIdx)

In [21]:
for key in dict3.keys():
    print("%s -- %d" %(key, len(dict3[key])))

seawifs_Rrs670 -- 181
Matchup_Rrs_unc_412 -- 181
Rrs_510 -- 181
Rrs_412 -- 181
insitu_Rrs510 -- 181
Matchup_Rrs_unc_510 -- 181
Rrs_unc_412 -- 181
seawifs_Rrs555 -- 181
Rrs_555 -- 181
insitu_Rrs670 -- 181
Rrs_unc_443 -- 181
Rrs_670 -- 181
Rrs_unc_510 -- 181
Matchup_Rrs_unc_670 -- 181
seawifs_Rrs490 -- 181
Rrs_443 -- 181
insitu_Rrs412 -- 181
Rrs_unc_670 -- 181
insitu_Rrs555 -- 181
Matchup_Rrs_unc_490 -- 181
insitu_Rrs490 -- 181
ID -- 181
Rrs_unc_490 -- 181
Rrs_490 -- 181
Matchup_Rrs_unc_443 -- 181
Matchup_Rrs_unc_555 -- 181
seawifs_Rrs412 -- 181
seawifs_Rrs443 -- 181
seawifs_Rrs510 -- 181
Rrs_unc_555 -- 181
insitu_Rrs443 -- 181


In [22]:
dfMupUncComp = pd.DataFrame(dict3)
dfMupUncComp.set_index('ID',inplace=True)

In [23]:
dfMupUncComp.columns

Index(['Matchup_Rrs_unc_412', 'Matchup_Rrs_unc_443', 'Matchup_Rrs_unc_490',
       'Matchup_Rrs_unc_510', 'Matchup_Rrs_unc_555', 'Matchup_Rrs_unc_670',
       'Rrs_412', 'Rrs_443', 'Rrs_490', 'Rrs_510', 'Rrs_555', 'Rrs_670',
       'Rrs_unc_412', 'Rrs_unc_443', 'Rrs_unc_490', 'Rrs_unc_510',
       'Rrs_unc_555', 'Rrs_unc_670', 'insitu_Rrs412', 'insitu_Rrs443',
       'insitu_Rrs490', 'insitu_Rrs510', 'insitu_Rrs555', 'insitu_Rrs670',
       'seawifs_Rrs412', 'seawifs_Rrs443', 'seawifs_Rrs490', 'seawifs_Rrs510',
       'seawifs_Rrs555', 'seawifs_Rrs670'],
      dtype='object')

In [26]:
dfMupUncComp.filter(regex='Rrs_unc',axis=1).columns

Index(['Matchup_Rrs_unc_412', 'Matchup_Rrs_unc_443', 'Matchup_Rrs_unc_490',
       'Matchup_Rrs_unc_510', 'Matchup_Rrs_unc_555', 'Matchup_Rrs_unc_670',
       'Rrs_unc_412', 'Rrs_unc_443', 'Rrs_unc_490', 'Rrs_unc_510',
       'Rrs_unc_555', 'Rrs_unc_670'],
      dtype='object')

In [27]:
dfMupUncComp.to_pickle('dfMupUncComp.pkl')

Index(['cruise_name', 'file_name', 'date_time', 'latitude', 'longitude',
       'seawifs_npixel', 'seawifs_tdiff', 'seawifs_solz', 'seawifs_senz',
       'seawifs_cv', 'seawifs_es_thresh', 'seawifs_windspeed', 'insitu_Rrs412',
       'insitu_Rrs443', 'insitu_Rrs490', 'insitu_Rrs510', 'insitu_Rrs555',
       'insitu_Rrs670', 'seawifs_Rrs412', 'seawifs_Rrs443', 'seawifs_Rrs490',
       'seawifs_Rrs510', 'seawifs_Rrs555', 'seawifs_Rrs670', 'L1_name'],
      dtype='object')

In [28]:
pickle.dump(dict1,open('dictRowQA.pkl','wb'))
pickle.dump(dict2,open('dictBoxQA.pkl','wb'))
pickle.dump(dict4,open('dictIdToBsnm.pkl','wb'))

In [372]:
flagList=['AtmFail','Land','BadAnc.','HiGlint','HiLt','HiSatZen','CoastZ','NegLw','StrayLight',
'CldIce','Cocco','TurbidW','HiSolZen','HiTau','LowLw','ChlFail','NavWarn','AbsAer',
'Tricho','MaxAerIter','ModGlint','ChlWarn','AtmWarn','DarkPixel','SeaIce','NavFail',
'Filter','SstWarn','SstFail','HiPol','spare','Ocean']
flagBitDict={e:i for i,e in enumerate(flagList)}

In [369]:
flagBits=0
pfd = dict.fromkeys(flagList)
for key in flagBitDict.keys():
    flagBit = 2**key
    if (flagBit & l2flags[rowcol]) > 0:
        pfd[flagBitDict[key]] = 'On'
    else:
        pfd[flagBitDict[key]] = 'Off'

In [364]:
(l2flags[boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]] & flagBits)>0

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]], dtype=bool)

In [359]:
popcount(l2flags[boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]])

array([[4, 4, 4, 4, 4],
       [5, 4, 4, 4, 4],
       [5, 5, 4, 2, 2],
       [5, 5, 5, 4, 2],
       [5, 5, 5, 5, 2]])

In [349]:
l2flags[boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]]

array([[1074791168, 1074791168, 1074791168, 1074791168, 1074791168],
       [1074791184, 1074791168, 1074791168, 1074791168, 1074791168],
       [1074791184, 1074791184, 1074791168,    1048832,    1048832],
       [1074791184, 1074791184, 1074791184, 1074791168,    1048832],
       [1074791184, 1074791184, 1074791184, 1074791184,    1048832]], dtype=int32)

In [267]:
bands = rrsDict.keys()
subRrsDict = dict.fromkeys(bands)
subRrsUncDict = dict.fromkeys(bands)
i,j = roColTpl
boxXRng = [i - 2,i + 3]
boxYRng = [j - 2,j + 3]
band='510'
subRrs = rrsDict[band][boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]]
subRrsUnc = rrsUncDict[band][boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]]

In [204]:
boxLims = [rowcol[0]-2,rowcol[0]+3,rowcol[1]-2,rowcol[1]+3]
abox=rrsDict['510'][boxLims[0]:boxLims[1],boxLims[2]:boxLims[3]]
bbox=rrsUncDict['510'][boxLims[0]:boxLims[1],boxLims[2]:boxLims[3]]

In [212]:
np.sqrt(np.sum(np.power(bbox.compressed(),2)))/bbox.compressed().size

1.3185633579269051e-05

In [218]:
subRrs,subRrsUnc=GetStats(rrsDict,rrsUncDict,rowcol)

In [224]:
for band in subRrs.keys():
    print('%s:Rrs=%.2e;Rrs_unc=%.2e' % (band,subRrs[band],subRrsUnc[band]))

510:Rrs=3.83e-03;Rrs_unc=1.32e-05
490:Rrs=5.93e-03;Rrs_unc=1.46e-05
443:Rrs=7.61e-03;Rrs_unc=1.74e-05
412:Rrs=8.06e-03;Rrs_unc=2.19e-05
670:Rrs=2.15e-04;Rrs_unc=6.18e-06
555:Rrs=1.87e-03;Rrs_unc=1.03e-05


In [228]:
dfSub.insitu_Rrs510,dfSub.seawifs_Rrs510

(id
 2113    0.003539
 Name: insitu_Rrs510, dtype: float64, id
 2113    0.00386
 Name: seawifs_Rrs510, dtype: float64)

In [109]:
uncLatRng, uncLonRng,uncLat,uncLon = GetUncLatLonRng(l2FileName)

In [328]:
rowcol,val = GetCntrPxlRowCol(uncLat,mUpLat,uncLon,mUpLon)

In [329]:
val

True

In [334]:
dfSub.latitude.values

array([ 31.183])

In [335]:
dfSub.seawifs_Rrs412.values

array([ 0.00796846])

In [336]:
dfSub.columns

Index(['cruise_name', 'file_name', 'date_time', 'latitude', 'longitude',
       'seawifs_npixel', 'seawifs_tdiff', 'seawifs_solz', 'seawifs_senz',
       'seawifs_cv', 'seawifs_es_thresh', 'seawifs_windspeed', 'insitu_Rrs412',
       'insitu_Rrs443', 'insitu_Rrs490', 'insitu_Rrs510', 'insitu_Rrs555',
       'insitu_Rrs670', 'seawifs_Rrs412', 'seawifs_Rrs443', 'seawifs_Rrs490',
       'seawifs_Rrs510', 'seawifs_Rrs555', 'seawifs_Rrs670', 'L1_name'],
      dtype='object')

In [337]:
dfSub.file_name.values

array(['S1999221200555.L2_MLAC.R0000020621_31N_30N_121W_120W.hdf_OCVAL.bz2'], dtype=object)

In [338]:
dfSub.L1_name.values

array(['S1999221200555.L1A_MLAC.R0000020621_31N_30N_121W_120W.hdf.bz2'], dtype=object)

In [331]:
mUpLat,mUpLon

(31.183000000000003, -120.935)

In [332]:
l2FileName

'/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/Matchups/L2/AllFiles/S1999221200555_silent.L2'

In [61]:
'''
dict1Labels = ['index','uncLatRng','mUpInLatRng','uncLonRng','mUpInLonRng']
dict2Labels = ['index','basename','boxCntrRowCol','boxValid','numValCells']
dict3Labels = ['index','basename',Rrs_412,Rrs_unc_412,...,Rrs_vvv, Rrs_unc_vvv]
'''

for idx in dfex.index.values:
    print(dfex.loc[idx,:])

cruise_name                                                   s000620w
file_name            S2000172164529.L2_MLAC.R0000021386_43N_43N_68W...
date_time                                          2000/06/20 14:30:00
latitude                                                        43.753
longitude                                                      -66.598
seawifs_npixel                                                     100
seawifs_tdiff                                                     9000
seawifs_solz                                                    21.371
seawifs_senz                                                    22.126
seawifs_cv                                                      0.0325
seawifs_es_thresh                                                  3.5
seawifs_windspeed                                                4.933
insitu_Rrs412                                               0.00248826
insitu_Rrs443                                               0.00247619
insitu

In [38]:
pickle.dump(mainDict,open('matchupsQADict_1.pkl','wb'))